In [9]:
import numpy as np
import pandas as pd
import geopandas as gpd
import pydeck as pdk

# 포스트 데이터 불러오기
df_post = pd.read_csv('../inputs/posts/garosugil.csv', index_col=0)

# 텍스트 없으면 drop
df_post = df_post.dropna()

In [7]:
# 상점 데이터 불러오기
gdf_store_garosugil = gpd.read_file('../inputs/stores/store_garosugil.geojson')

# 상호명 없는 상점은 삭제
gdf_store_garosugil = gdf_store_garosugil.dropna(subset='상호명')

# 상호명과 geometry만 남기고 다 삭제 (현재 필요없는 컬럼들)
gdf_store_garosugil = gdf_store_garosugil.loc[:,['상호명', 'geometry']]

# 상호명: store_name으로 변경
gdf_store_garosugil = gdf_store_garosugil.rename(columns={'상호명':'store_name'})

In [8]:
gdf_store_garosugil.shape

(7348, 2)

In [10]:
def counting_post(store_name):
    '''
    상점 이름을 input으로 받아 posting 갯수를 세주는 함수

    input:
        store_name (string): 상호명
    output:
        posting_count (int): 포스팅 갯수

    '''

    # 상호명이 한글자이면 그냥 0개로 취급
    if len(store_name) == 1:
        return 0

    # 포스트 중에서 상점 이름이 포함되어 있으면 좌표 추가
    posting_count = df_post.loc[df_post.loc[:,'text'].str.contains(store_name, regex=False)].shape[0]

    return posting_count


In [11]:
gdf_store_garosugil.loc[:,'posting_count'] = gdf_store_garosugil.loc[:,'store_name'].apply(counting_post)

In [12]:
# 포스팅 갯수 기준으로 정렬
gdf_store_garosugil = gdf_store_garosugil.sort_values(by='posting_count', ascending=False)

In [13]:
for i, row in gdf_store_garosugil.iterrows():
    print(row['store_name'], row['posting_count'])
    if row['posting_count'] == 0:
        break

압구정 357
만나 186
오픈 97
라인 96
라인 96
사이 93
사이 93
리뷰 89
랜드 79
프레임 77
신사동술집 72
일리 70
이나 69
부부 67
포인트 64
데일리 62
화이트 55
베이 55
와인사랑 46
제이 42
크림 41
크림 41
크림 41
지나 40
하우 39
카이 37
카이 37
자이 32
압구정술집 32
지니 31
이즈 31
그레이 30
소스 28
아가 27
컴퍼니 27
힐링 24
강남사 23
제이윤뷰티 23
메이드 22
피오스튜디오 21
시즌 20
드림 19
미나 19
온다 17
가보 17
비프렌즈 17
디데이성형외과의원 16
관계 16
파인 15
유스튜디오 15
조각집 15
바른 14
해가 14
와이 14
센스 14
삐땅기린뷰티 12
정원 12
헤어바이이은 12
베이지 12
오프닝 11
인다 11
프로젝트 11
인트로 11
도우 11
루이 11
브리온 10
상상 10
바스튜디오 10
동화 10
리온 10
바누아즈 9
미로 9
이에스 9
이태 9
제이에스 9
컬렉션 9
도시 9
현대 9
틴트 9
현대 9
채율 9
원픽 9
우와 9
보카 9
릴리 8
뉴어스 8
에스프레소 8
보일 8
원네일 8
노하우 8
미슐랭 8
노하우 8
베스트 8
오버레이 8
헤버구떼 8
프랑 8
베스트 8
에스프레소 8
사라 7
페어링 7
준오헤어 7
만남 7
스콘 7
쉬작 7
성패션 7
르메 7
더브리 7
코스메틱 7
이레 7
스콘 7
스포츠 7
준오헤어 7
세심 7
우연 7
러쉬 6
토마토 6
불리 6
씨티 6
페타테 6
마일스톤 6
다교 6
라이프 6
리나 6
조화원 6
언노운 5
옥토 5
파치노 5
파치노 5
말랑 5
이조 5
마사 5
소각 5
다이아 5
자매의부엌 5
레오 5
뷰티건강연구소 5
미우 4
우니도 4
올리브 4
버드맨 4
유희 4
올드페리도넛 4
콜키지 4
다오 4
게임 4
신사전 4
타래 4
디바 4
강남# 4
진도켄 4
온다네일 4
퓨어 4
온다네일 4
민트 4
이니셜 4
코튼 4
다래 3
닥터브로우 3
밀떡팩토리 3
유후네일 3

In [14]:
list_stopwords = [
    '압구정',
'만나',
'오픈',
'라인',
'라인',
'사이',
'사이',
'리뷰',
'랜드',
'프레임',
'신사동술집',
'일리',
'이나',
'부부',
'포인트',
'데일리',
'화이트',
'베이',
'와인사랑',
'제이',
'크림',
'크림',
'크림',
'지나',
'하우',
'카이',
'카이',
'자이',
'압구정술집',
'지니',
'이즈',
'그레이',
'소스',
'아가',
'컴퍼니',
'힐링',
'강남사'
]

In [15]:
# list_stopwords 안에 있는 상점들은 우선 데이터프레임에서 제외
gdf_store_garosugil = gdf_store_garosugil.loc[~gdf_store_garosugil.loc[:,'store_name'].isin(list_stopwords)]

# 포스팅 수가 0인 것들도 데이터프레임에서 제외
gdf_store_garosugil = gdf_store_garosugil.loc[gdf_store_garosugil.loc[:,'posting_count']>0]

In [16]:
gdf_store_garosugil

,store_name,geometry,posting_count
6971,제이윤뷰티,POINT (127.02400 37.51931),23
4488,메이드,POINT (127.02561 37.52491),22
5848,피오스튜디오,POINT (127.02348 37.51945),21
3415,시즌,POINT (127.02758 37.52084),20
5569,드림,POINT (127.02882 37.52457),19
...,...,...,...
4401,그람,POINT (127.02962 37.51840),1
99,강남주막다방,POINT (127.02335 37.51901),1
4693,연경,POINT (127.02068 37.51876),1
3202,육회담,POINT (127.02068 37.51876),1


In [17]:
# pydeck layer 설정

scatterplot_layer = pdk.Layer(
    'ScatterplotLayer',
    gdf_store_garosugil,
    pickable=True,
    opacity=0.3,
    stroked=False,
    filled=True,
    radius_scale=2,
    get_position='geometry.coordinates',
    get_radius='posting_count',
    get_fill_color=[255,140,0]
)

In [19]:
# Set the viewport location
view_state = pdk.ViewState(latitude=37.5210, longitude=127.0228, zoom=15, bearing=0, pitch=0)

# Render
r = pdk.Deck(layers=[scatterplot_layer], initial_view_state=view_state, tooltip={"text": "{store_name}\n{posting_count}"})
r.to_html("../output/maps/garosugil/scatterplot_layer.html")

In [21]:
heatmap_layer = pdk.Layer(
    "HeatmapLayer",
    data=gdf_store_garosugil,
    opacity=0.5,
    get_position='geometry.coordinates',
    aggregation=pdk.types.String("MEAN"),
    get_weight="posting_count",
)

In [22]:
r = pdk.Deck(
    layers=[heatmap_layer],
    initial_view_state=view_state
)

r.to_html("../output/maps/garosugil/heatmap_layer.html")